In [57]:
import numpy as np
import unittest
import matplotlib.pyplot as plt

In [91]:
class Lagger:
    def __init__(self, b=2, o=4):
        self.b = b
        self.o = o

    def __call__(self, t, n):
        l0 = np.sqrt(self.o) * np.exp(-self.b * t / 2)
        l1 = np.sqrt(self.o) * (1 - self.o * t) * np.exp(-self.b * t / 2)
        if n == 0:
            return l0
        elif n == 1:
            return l1
        else:
            for i in range(2, n + 1):
                l2 = (2 * i - 1 - self.o * t) * l1 / i - (i - 1) * l0 / i
                l0, l1 = l1, l2
            return l2
        
    def tabulate_lagger(self, n, T, k):
        h = T / k
        x = [i*h for i in range(k+1)]
        y = [self(x1,n) for x1 in x]
        return x, y

    @staticmethod
    def print_tabulation(x, y):
        for i in range(len(x)):
            print(round(x[i], 5), y[i], sep='\t')
            
    def experiment(self, T, eps=1e-3, N=20):
        t = 0
        h = 1/10
        while t <= T:
            flag = True
            for n in range(1, N + 1):
                if abs(self(t, n)) > eps:
                    flag = False
                    break
            if flag:
                return t
            else:
                t += h

In [92]:
class Integral:
    def __init__(self, func, a, b):
        self.func=func
        self.a=a
        self.b=b  
   
    def integral(self, k):
        in1 = 0
        h = (self.b - self.a) / k
        for i in range(k):
            x = h * i + h / 2
            in1 += self.func(x)
        return in1 * h

    def integral_main(self, eps=0.00001):
        k = 10
        prev = self.integral(k)
        k *= 2
        cur = self.integral(k)
        while abs(prev - cur) > eps:
            k *= 2
            prev, cur = cur, self.integral(k)
        return cur

In [94]:
class Transformation:
    def __init__(self, func, T, N=10):
        self.func=func
        self.T=T
        self.N=N
        self.lag=Lagger(2,4)
    
    def lagger_transformation(self):
        fk=[]
        for i in range(self.N+1):
            def int_func(t):
                return self.func(t)*self.lag(t,i)*np.exp(-2*t)
            integ=Integral(int_func,0,self.T)
            fk.append(integ.integral_main())
        return fk 
    
    @staticmethod
    def reverse_lagger(transform_res,x):
        sum=0
        lag=Lagger(2,4)
        for i in range(len(transform_res)):
            sum+=transform_res[i]*lag(x,i)
        return sum

In [88]:
Lag=Lagger(2,4)
Lag(3,30)

3.463236074526919

In [32]:
x_vals, y_vals = Lag.tabulate_lagger(n=6, T=10, k=10)
Lagger.print_tabulation(x_vals, y_vals)

0.0	1.9999999999999998
1.0	-1.3570663829879872
2.0	3.254061699155888
3.0	-8.503631277231161
4.0	0.16687582098624532
5.0	68.77048171577698
6.0	162.3414120568452
7.0	228.83378017744135
8.0	246.3696729833059
9.0	223.06198678863245
10.0	178.63206688559174


In [95]:
exp=Lag.experiment(100)
print(exp)

79.09999999999978


In [34]:
def func(t):
    return Lag(t,2)*Lag(t,3)*np.exp(-2*t)

In [71]:
integral_res=Integral(func,4,27)
print(integral_res.integral_main())

-1.2612314372569622e-06


In [39]:
def f_cos(t):
    if t>=2*np.pi:
        return 0
    else:
        return np.cos(t-np.pi/4)+1-np.sin(t+np.pi/6)

In [41]:
transform_val=Transformation(f_cos,exp)
transform_res=transform_val.lagger_transformation()
print(transform_res)

[0.7591458311093235, -0.18601182936288674, 0.04109031538239029, -0.018732421028574066, 0.012394863182049813, -0.005453122986364942, 0.0008142670798417326, 0.0008139930946952234, -0.0008291950825886384, 0.00022703627424913006, 0.0003420671473524219]


In [67]:
reverse_res=Transformation.reverse_lagger(transform_res,5)
print(reverse_res)

1.2163453848995567


In [89]:
class TestLagger(unittest.TestCase):
    
    def testCall(self):
        Lag=Lagger(2,4)
        self.assertEqual(Lag(1,20),-0.11094613260729669)
        self.assertEqual(Lag(2,20), 1.7779257824923866)
        self.assertEqual(Lag(3,30), 3.463236074526919)
    
    def testExperiment(self):
        Lag=Lagger(2,4)
        self.assertEqual(Lag.experiment(100),79.09999999999978)
        self.assertEqual(Lag.experiment(10),None)
        self.assertEqual(Lag.experiment(90),79.09999999999978)
    
    def testIntegral(self):
        integral_res=Integral(func,0,100)
        self.assertEqual(integral_res.integral_main(),-1.4901121472831865e-06)
        integral_res=Integral(func,0,10)
        self.assertEqual(integral_res.integral_main(),-9.536431577891995e-07)
        integral_res=Integral(func,4,27)
        self.assertEqual(integral_res.integral_main(),-1.2612314372569622e-06)
    
    def testTransform(self):
        exp=79.09999999999978
        transform_val=Transformation(f_cos,exp)
        self.assertEqual(transform_val.lagger_transformation(),[0.7591458311093235, -0.18601182936288674, 0.04109031538239029, -0.018732421028574066, 0.012394863182049813, -0.005453122986364942, 0.0008142670798417326, 0.0008139930946952234, -0.0008291950825886384, 0.00022703627424913006, 0.0003420671473524219])
    
    def testReverse(self):
        self.assertEqual(Transformation.reverse_lagger(transform_res,10),-0.14863486648633106)
        self.assertEqual(Transformation.reverse_lagger(transform_res,9),-0.7464518658693642)
        self.assertEqual(Transformation.reverse_lagger(transform_res,5),1.2163453848995567)

In [90]:
unittest.main(argv=[''], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 107.114s

OK
